In [ ]:
import os
import json
import re
JSON_FILE_PATH = "top_250_movies.json"
SUMMARIES_DIR = "ozetler"

def create_safe_filename(title):
    safe_name = re.sub(r'[\\/*?:"<>|]', "_", title)
    # Sonuna .txt uzantısını ekle
    return f"{safe_name}.txt"

def analyze_summaries():

    try:
        with open(JSON_FILE_PATH, "r", encoding="utf-8") as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"HATA: '{JSON_FILE_PATH}' dosyası bulunamadı. Lütfen dosya yolunu kontrol edin.")
        return
    except json.JSONDecodeError:
        print(f"HATA: '{JSON_FILE_PATH}' dosyası geçerli bir JSON formatında değil.")
        return

    master_movie_list = []
    if isinstance(data, list):
        for movie in data:
            if "title" in movie:
                master_movie_list.append(movie["title"])
    
    if not master_movie_list:
        print("JSON dosyasından hiç film başlığı okunamadı.")
        return

    print(f"JSON dosyasından toplam {len(master_movie_list)} film başlığı başarıyla okundu.")
    print("-" * 40)

    if not os.path.exists(SUMMARIES_DIR):
        print(f"HATA: '{SUMMARIES_DIR}' klasörü bulunamadı. Lütfen klasör adını kontrol edin.")
        return
        
    actual_files = os.listdir(SUMMARIES_DIR)
    print(f"'{SUMMARIES_DIR}' klasöründe toplam {len(actual_files)} dosya bulundu.")
    print("-" * 40)
.
    missing_files = []
    zero_byte_files = []
    no_extension_files = []
    
    expected_filenames = set()
    for title in master_movie_list:
        expected_filename = create_safe_filename(title)
        expected_filenames.add(expected_filename)
        if expected_filename not in actual_files:
            missing_files.append(title)
            
    for filename in actual_files:
        full_path = os.path.join(SUMMARIES_DIR, filename)
        
        if not filename.endswith('.txt'):
            no_extension_files.append(filename)
        
        try:
            if os.path.getsize(full_path) == 0:
                zero_byte_files.append(filename)
        except OSError as e:
            print(f"Dosya boyutu okunurken hata: {filename} - {e}")

    print("\n✅ ANALİZ SONUÇLARI ✅\n")

    if missing_files:
        print(f"--- ❌ EKSİK VEYA İSMİ HATALI ÖZET DOSYALARI ({len(missing_files)} adet) ---")
        for i, title in enumerate(missing_files, 1):
            print(f"{i}. {title}")
    else:
        print("--- ✅ Tüm filmlerin özeti oluşturulmuş görünüyor. ---")

    print("\n" + "-" * 40 + "\n")

    if zero_byte_files:
        print(f"--- ❌ 0 KB BOYUTUNDA (BOŞ) DOSYALAR ({len(zero_byte_files)} adet) ---")
        for i, filename in enumerate(zero_byte_files, 1):
            print(f"{i}. {filename}")
    else:
        print("--- ✅ 0 KB boyutunda dosya bulunamadı. ---")

    print("\n" + "-" * 40 + "\n")
    with open("missfilmler.txt", "w", encoding="utf-8") as f:
            for film in missing_files:
                f.write(film + "\n")

    if no_extension_files:
        print(f"--- ❌ '.txt' UZANTISI OLMAYAN DOSYALAR ({len(no_extension_files)} adet) ---")
        for i, filename in enumerate(no_extension_files, 1):
            print(f"{i}. {filename}")
    else:
        print("--- ✅ Tüm dosyalar '.txt' uzantısına sahip. ---")


if __name__ == "__main__":
    analyze_summaries()
    

JSON dosyasından toplam 250 film başlığı başarıyla okundu.
----------------------------------------
'ozetler' klasöründe toplam 238 dosya bulundu.
----------------------------------------

✅ ANALİZ SONUÇLARI ✅

--- ❌ EKSİK VEYA İSMİ HATALI ÖZET DOSYALARI (37 adet) ---
1. The Lord of the Rings: The Return of the King
2. The Lord of the Rings: The Fellowship of the Ring
3. The Lord of the Rings: The Two Towers
4. Spider-Man: Into the Spider-Verse
5. Primal: Tales of Savagery
6. A Silent Voice: The Movie
7. Gabriel's Inferno: Part II
8. Gabriel's Inferno: Part III
9. Spider-Man: Across the Spider-Verse
10. Léon: The Professional
11. Violet Evergarden: The Movie
12. Neon Genesis Evangelion: The End of Evangelion
13. Taylor Swift: Reputation Stadium Tour
14. Avengers: Endgame
15. Avengers: Infinity War
16. Evangelion: 3.0+1.0 Thrice Upon a Time
17. Ultraman: Rising
18. Justice League Dark: Apokolips War
19. Puss in Boots: The Last Wish
20. Demon Slayer -Kimetsu no Yaiba- The Movie: Mugen Tr

In [6]:
import sqlite3

def create_empty_db_with_schema(db_path: str, schema_sql_file: str):

    with open(schema_sql_file, 'r') as f:
        schema_sql = f.read()
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.executescript(schema_sql)
    conn.commit()
    conn.close()
    print(f"'{db_path}' üzerinde şema oluşturuldu.")

def verify_sql_syntax(db_path: str, sql_query: str) -> tuple[bool, str]:
    if not sql_query.strip().upper().startswith("SELECT"):
        return (False, "Güvenlik hatası: Sadece SELECT sorguları doğrulanabilir.")

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        explained_query = f"EXPLAIN QUERY PLAN {sql_query}"
        cursor.execute(explained_query)
        conn.close()
        return (True, "Sorgu geçerli.")
    except sqlite3.Error as e:
        return (False, f"Sorgu geçersiz: {e}")

DB_PATH = 'movies_final_25.db'

valid_query = "SELECT T1.title FROM movies AS T1 INNER JOIN movie_genres AS T2 ON T1.imdb_id = T2.movie_imdb_id INNER JOIN genres AS T3 ON T2.genre_id = T3.id WHERE T3.name = 'Action' AND T1.original_language = 'en' LIMIT 5"
is_valid, message = verify_sql_syntax(DB_PATH, valid_query)
print(f"'{valid_query[:30]}...': {is_valid}, Mesaj: {message}")

invalid_query = "SELECT title FROM movies"
is_valid, message = verify_sql_syntax(DB_PATH, invalid_query)
print(f"'{invalid_query[:30]}...': {is_valid}, Mesaj: {message}")

'SELECT T1.title FROM movies AS...': True, Mesaj: Sorgu geçerli.
'SELECT title FROM movies...': True, Mesaj: Sorgu geçerli.


In [9]:
import json
import sqlite3
import re

TXT_PATH = "verii.txt"
DB_PATH = "movies_final.db"

valid_queries = []
invalid_queries = []

conn = sqlite3.connect(DB_PATH)
conn.execute("PRAGMA foreign_keys = OFF;")
cursor = conn.cursor()

sql_pattern = re.compile(r"^\s*(SELECT|INSERT|UPDATE|DELETE|WITH)\b", re.IGNORECASE)

with open(TXT_PATH, "r", encoding="utf-8") as f:
    for line_no, line in enumerate(f, start=1):
        try:
            data = json.loads(line.strip())
        except json.JSONDecodeError:
            continue
        
        sql_text = None
        if isinstance(data.get("output"), str) and sql_pattern.match(data["output"]):
            sql_text = data["output"]

        elif isinstance(data.get("input"), dict):
            for val in data["input"].values():
                if isinstance(val, str) and sql_pattern.match(val):
                    sql_text = val
                    break

        if sql_text:
            try:
                cursor.execute(sql_text)
                valid_queries.append((line_no, sql_text))
            except Exception as e:
                invalid_queries.append((line_no, sql_text, str(e)))

print("\n✅ Geçerli SQL sorguları:")
for ln, q in valid_queries:
    print(f"  [Satır {ln}] {q}")

print("\n❌ Hatalı SQL sorguları:")
for ln, q, err in invalid_queries:
    print(f"  [Satır {ln}] {q}\n    Hata: {err}")

conn.close()



✅ Geçerli SQL sorguları:
  [Satır 3] SELECT runtime FROM movies WHERE title = '12 Angry Men'
  [Satır 6] SELECT T1.title FROM movies AS T1 WHERE T1.imdb_id IN (SELECT T2.movie_imdb_id FROM movie_crew AS T2 INNER JOIN crew_members AS T3 ON T2.crew_member_id = T3.id WHERE T3.name = 'Quentin Tarantino' AND T2.job = 'Director') AND T1.imdb_id IN (SELECT T4.movie_imdb_id FROM movie_genres AS T4 INNER JOIN genres AS T5 ON T4.genre_id = T5.id WHERE T5.name = 'Crime') AND T1.imdb_id IN (SELECT movie_imdb_id FROM movie_cast INNER JOIN actors ON movie_cast.actor_id = actors.id WHERE actors.name = 'Samuel L. Jackson') AND T1.imdb_id IN (SELECT movie_imdb_id FROM movie_cast INNER JOIN actors ON movie_cast.actor_id = actors.id WHERE actors.name = 'Uma Thurman')
  [Satır 9] SELECT T1.title FROM movies AS T1 INNER JOIN movie_cast AS T2 ON T1.imdb_id = T2.movie_imdb_id INNER JOIN actors AS T3 ON T2.actor_id = T3.id WHERE T3.name = 'Al Pacino' AND T1.imdb_id NOT IN (SELECT T4.movie_imdb_id FROM movie_

In [ ]:
import sqlite3
import os

def get_compressed_schema(db_path: str) -> str:
    if not os.path.exists(db_path):
        return f"Hata: Veritabanı dosyası bulunamadı -> {db_path}"

    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()


        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
        tables = cursor.fetchall()

        all_table_schemas = []
        for table_name_tuple in tables:
            table_name = table_name_tuple[0]
            
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            
            column_names = [column[1] for column in columns]
            
            table_schema_str = f"{table_name}({','.join(column_names)})"
            all_table_schemas.append(table_schema_str)

        return "; ".join(all_table_schemas)

    except sqlite3.Error as e:
        return f"Veritabanı hatası: {e}"
    finally:
        if conn:
            conn.close()

dummy_db_path = "movies_final.db"

compressed_schema = get_compressed_schema(dummy_db_path)

print("--- SIKIŞTIRILMIŞ VERİTABANI ŞEMASI ---")
print(compressed_schema)

os.remove(dummy_db_path)

--- SIKIŞTIRILMIŞ VERİTABANI ŞEMASI ---
movies(imdb_id,title,original_title,overview,tagline,release_date,runtime,original_language,adult); genres(id,name); production_companies(id,name); countries(id,name); spoken_languages(id,name); actors(id,name); crew_members(id,name); movie_genres(movie_imdb_id,genre_id); movie_companies(movie_imdb_id,company_id); movie_countries(movie_imdb_id,country_id); movie_languages(movie_imdb_id,language_id); movie_cast(movie_imdb_id,actor_id,character); movie_crew(movie_imdb_id,crew_member_id,job)


In [ ]:
import json


INPUT_DOSYA_ADI = "verii.txt"

OUTPUT_DOSYA_ADI = "guncellenmis_verim.txt"


YENI_SEMA_METNI = "Tablo: movies(imdb_id,title,original_title,overview,tagline,release_date,runtime,original_language,adult); genres(id,name); production_companies(id,name); countries(id,name); spoken_languages(id,name); actors(id,name); crew_members(id,name); movie_genres(movie_imdb_id,genre_id); movie_companies(movie_imdb_id,company_id); movie_countries(movie_imdb_id,country_id); movie_languages(movie_imdb_id,language_id); movie_cast(movie_imdb_id,actor_id,character); movie_crew(movie_imdb_id,crew_member_id,job)"

def standardize_db_schema(input_path, output_path, new_schema):

    try:
        lines_processed = 0
        lines_changed = 0

        with open(input_path, 'r', encoding='utf-8') as infile, \
             open(output_path, 'w', encoding='utf-8') as outfile:

            for line in infile:
                if not line.strip():
                    continue
                
                lines_processed += 1
                try:
                    data = json.loads(line)

                    if 'input' in data and isinstance(data.get('input'), dict):
                        if 'db_schema' in data['input']:
                            data['input']['db_schema'] = new_schema
                            lines_changed += 1
                    

                    updated_line = json.dumps(data, ensure_ascii=False)
                    outfile.write(updated_line + '\n')

                except json.JSONDecodeError:

                    print(f"Uyarı: {lines_processed}. satır JSON formatında değil. Olduğu gibi kopyalanıyor.")
                    outfile.write(line)

        print("\nİşlem başarıyla tamamlandı!")
        print(f"-> Toplam {lines_processed} satır işlendi.")
        print(f"-> {lines_changed} satırda 'db_schema' alanı güncellendi.")
        print(f"-> Sonuçlar '{output_path}' dosyasına kaydedildi.")

    except FileNotFoundError:
        print(f"HATA: '{input_path}' adında bir dosya bulunamadı. Dosya adını kontrol edin.")
    except Exception as e:
        print(f"Beklenmedik bir hata oluştu: {e}")


if __name__ == "__main__":
    standardize_db_schema(INPUT_DOSYA_ADI, OUTPUT_DOSYA_ADI, YENI_SEMA_METNI)


İşlem başarıyla tamamlandı!
-> Toplam 225 satır işlendi.
-> 65 satırda 'db_schema' alanı güncellendi.
-> Sonuçlar 'guncellenmis_verim.txt' dosyasına kaydedildi.
